# End to End Pure Streaming Data-Pipeline for Rent Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read rent state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "rent_streamingDF" --> "results_df" --> "rent_data"

In [3]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [4]:
# Schema  for Rent
rent_schema = StructType([
            StructField("Rent_id", IntegerType(), False),
            StructField("Rent_fee", StringType(), True),
            StructField("Late_fee", StringType(), False),
            StructField("Due_date", TimestampType(), True),
            StructField("Lease_id", IntegerType(), True),
            StructField("Pay_id", IntegerType(), True),
            StructField("event_time", TimestampType(), True)])

rent_udf_schema = StructType([
            StructField("Rent_fee", StringType(), True),
            StructField("Late_fee", StringType(), False),
            StructField("Due_date", TimestampType(), True),
            StructField("Lease_id", IntegerType(), True),
            StructField("Pay_id", IntegerType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get rent csv files as a streaming "rent_streamingDF" and process it on the fly and get transformed stream "rent_df"
###### Objective: (incoming csv files) --> "rent_streamingDF" --> "rent_df"

In [6]:
# Get rent Steaming DataFrame from csv files

# streaming starts here by reading the input files 
rent_Path = "/FileStore/apartment/rent/inprogress/"
rent_streamingDF = (
  spark
    .readStream
    .schema(rent_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(rent_Path)
)
# Clear invalid rows
rent_df = rent_streamingDF.select("*").where("Rent_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(rent_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
# transformer.remove_special_chars(columns=['rent_name', 'Address_line_1', 'City', 'Post_code', 'Region'])

##### This function parses the corresponding columns into a single column

In [8]:
def my_fun(Rent_fee, Late_fee, Due_date, Lease_id, Pay_id, event_time):
  return zip(Rent_fee, Late_fee, Due_date, Lease_id, Pay_id, event_time)

udf_Fun = udf(my_fun, ArrayType(rent_udf_schema))

In [9]:
intermediate_df = ( rent_df.withWatermark("event_time", "10 seconds")
            .groupBy("Rent_id")
            .agg(F.collect_list("Rent_fee").alias("Rent_fee"),
                 F.collect_list("Late_fee").alias("Late_fee"),
                 F.collect_list("Due_date").alias("Due_date"),
                 F.collect_list("Lease_id").alias("Lease_id"),
                 F.collect_list("Pay_id").alias("Pay_id"),
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Rent_id", 
                    F.explode(udf_Fun(F.column("Rent_fee"), 
                                      F.column("Late_fee"), 
                                      F.column("Due_date"), 
                                      F.column("Lease_id"), 
                                      F.column("Pay_id"),
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [11]:
results_df = (intermediate_df
              .select("Rent_id", 
                      "data.Rent_fee", 
                      "data.Late_fee", 
                      "data.Due_date", 
                      "data.Lease_id", 
                      "data.Pay_id", 
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Rent_id")

##### Display final result
###### This result shows the latest state of all the unique rent_id

In [13]:
display(results_df)

##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [16]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/rent/_checkpoints/streaming-agg")
 .start("/delta/apartment/rent_data"))

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe wil always be Up-To-Date

In [18]:
rent_data = spark.read.format("delta").load("/delta/apartment/rent_data").orderBy("Rent_id")

In [19]:
display(rent_data)

### Do Some Live Streaming Graphs

In [21]:
rent_data_stream = spark.readStream.format("delta").load("/delta/apartment/rent_data")

In [22]:
display(rent_data_stream.groupBy("Lease_id").count())